<a href="https://colab.research.google.com/github/godpeny/laboratory/blob/master/Study/NLP_Using_Deep_Learning/Bidirectional_Encoder_Representation_From_Transformer/multiple_classification_using_kobert_with_kornli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Default Setting For Using TPU in Google Colab

In [1]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import urllib.request
import csv

from sklearn import preprocessing

import keras
from transformers import BertTokenizer, TFBertModel
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# 훈 련 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")
# 검 증 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")
# 테 스 트 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

train_xnli = pd.read_csv('multinli.train.ko.tsv', sep='\t', quoting=csv.QUOTE_NONE)
train_snli = pd.read_csv('snli_1.0_train.ko.tsv', sep='\t', quoting=csv.QUOTE_NONE)
val_data = pd.read_csv('xnli.dev.ko.tsv', sep='\t', quoting=csv.QUOTE_NONE)
test_data = pd.read_csv('xnli.test.ko.tsv', sep='\t', quoting=csv.QUOTE_NONE)

print(val_data.tail(5))
print(test_data.tail(5))

                                           sentence1                sentence2  \
2485  피스티는 피즐처럼 중간 영어의 피스틴으로 시작되어 방귀를 뀌기 위해 주먹을 쥐었다.  Fiesty는 100년 동안 존재해 왔다.   
2486  피스티는 피즐처럼 중간 영어의 피스틴으로 시작되어 방귀를 뀌기 위해 주먹을 쥐었다.      Fiesty는 주먹질과는 무관하다.   
2487                진술이 더 나은 반면, 대답은 완성의 정신적 그림을 준다.      진술은 더 자세한 내용을 알려준다.   
2488                진술이 더 나은 반면, 대답은 완성의 정신적 그림을 준다.         진술이 더 나은 것은 아니다.   
2489                진술이 더 나은 반면, 대답은 완성의 정신적 그림을 준다.                진술이 더 좋다.   

         gold_label  
2485        neutral  
2486  contradiction  
2487        neutral  
2488  contradiction  
2489     entailment  
                                              sentence1  \
5005  데이비슨은 스콘의 발음을 '뼈'와 운을 맞추기 위해 채택해서는 안 된다. 어쨌든 그...   
5006  데이비슨은 스콘의 발음을 '뼈'와 운을 맞추기 위해 채택해서는 안 된다. 어쨌든 그...   
5007            25달러에 20만 단어의 평균 소설은 달러당 8,000단어로 적용된다.   
5008            25달러에 20만 단어의 평균 소설은 달러당 8,000단어로 적용된다.   
5009            25달러에 20만 단어의 평균 소설은 달러당 8,000단어로 적용된다.   

                          

# Data Preprocessing

In [4]:
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1) # shuffle

train_data.head(5)

<ipython-input-4-bf5aeb3582dd>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_snli.append(train_xnli)


,sentence1,sentence2,gold_label
35054,한 남자가 산 근처를 걷는다.,산을 향해 하이킹하는 남자.,neutral
243742,다른 종류의 탑을 사용하는 이 게임의 또 다른 파생상품은 상대의 윗부분을 회전 영역...,게임의 또 다른 파생물은 다른 상단과 매우 다른 규칙을 가지고 있습니다.,neutral
85108,한 남자가 반바지를 입고 벤치에 앉아 있는 동안 한 여성이 뉴욕의 보도를 걷고 있다.,한 남자가 샌프란시스코를 걸어 내려간다.,contradiction
441857,코트 위에 검은색과 금색 옷을 입은 남자가 흑백 말을 탄다.,말이 질주하고 있다.,neutral
252964,물속에서 노를 젓는 두 남자.,밖에 있는 사람들,entailment


In [5]:
def drop(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)

  return df

In [6]:
train_data_processed = drop(train_data)
val_data_processed = drop(val_data)
test_data_processed = drop(test_data)

print(len(train_data_processed))
print(len(val_data_processed))
print(len(test_data_processed))

941814
2490
5010


# Tokenizing

In [7]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
max_seq_len = 128

def tokenize(sentences_1, sentences_2):
  tokenized_sentences, attention_masks, token_types = [], [], []

  for sen1, sen2 in zip(sentences_1, sentences_2):
    results = tokenizer.encode_plus(sen1, sen2, max_length=max_seq_len, pad_to_max_length=True)

    tokenized_sentences.append(results.input_ids)
    attention_masks.append(results.attention_mask)
    token_types.append(results.token_type_ids)

  tokenized_sentences_np = np.array(tokenized_sentences, dtype=int)
  attention_masks_np = np.array(attention_masks, dtype=int)
  token_types_np = np.array(token_types, dtype=int)

  return (tokenized_sentences_np, attention_masks_np, token_types_np)

In [8]:
X_train = tokenize(train_data_processed.sentence1, train_data_processed.sentence2) # takes long time - approximaely 9 min in tpu colab

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs 

In [9]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]
print(' 단 어 에 대 한 정 수 인 코 딩 :',input_id)
print(' 어 텐 션 마 스 크 :',attention_mask)
print(' 세 그 먼 트 인 코 딩 :',token_type_id)
print(' 각 인 코 딩 의 길 이 :', len(input_id))
print(' 정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

 단 어 에 대 한 정 수 인 코 딩 : [   2 1891 3997 2116 1235 6328 2138  571 2259 2062   18    3 1235 2069
 4985 4899 2883 2205 2259 3997   18    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
 어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 세 그 먼 트 인 코 딩 : [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 

In [10]:
X_val = tokenize(val_data_processed.sentence1, val_data_processed.sentence2)
X_test = tokenize(test_data_processed.sentence1, test_data_processed.sentence2)

In [11]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]
print(' 단 어 에 대 한 정 수 인 코 딩 :',input_id)
print(' 어 텐 션 마 스 크 :',attention_mask)
print(' 세 그 먼 트 인 코 딩 :',token_type_id)
print(' 각 인 코 딩 의 길 이 :', len(input_id))
print(' 정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

 단 어 에 대 한 정 수 인 코 딩 : [   2 1891 3997 2116 1235 6328 2138  571 2259 2062   18    3 1235 2069
 4985 4899 2883 2205 2259 3997   18    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
 어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 세 그 먼 트 인 코 딩 : [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 

In [12]:
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]
print(' 단 어 에 대 한 정 수 인 코 딩 :',input_id)
print(' 어 텐 션 마 스 크 :',attention_mask)
print(' 세 그 먼 트 인 코 딩 :',token_type_id)
print(' 각 인 코 딩 의 길 이 :', len(input_id))
print(' 정 수 인 코 딩 복 원 :',tokenizer.decode(input_id))

 단 어 에 대 한 정 수 인 코 딩 : [    2  3673   636  2116  1041  2371  2062    16     6  4122    16  1535
  1458 10283    18     6     3   636  2259  3741  4942  2116   636  2138
  4105  2223  2155  6000  4122  2170  2318  4117  2138   572  2359  2062
    18     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
 어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [13]:
# Tokenizing Labels
train_label = train_data_processed['gold_label'].tolist()
val_label = val_data_processed['gold_label'].tolist()
test_label = test_data_processed['gold_label'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

print(idx_encode.classes_)

# change label to numbers
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

print(y_train[:10])

['contradiction' 'entailment' 'neutral']
[2 2 0 2 1 1 1 1 2 1]


# Modeling

In [14]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)

    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                            activation='softmax',
                                            name='classifier')
  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)

    return prediction

In [15]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

early_stopping = EarlyStopping(monitor="val_accuracy",
                               min_delta=0.001,
                               patience=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [16]:
# cut train datas due to memory insufficient issue in colab
sample_num = 50000

data, mask, ids = X_train
X_train_light = (data[:sample_num], mask[:sample_num], ids[:sample_num])
y_train_light = y_train[:sample_num]

In [17]:
history = model.fit(X_train_light, y_train_light, epochs=2, batch_size=32, validation_data = (X_val, y_val),callbacks = [early_stopping])

Epoch 1/2
1563/1563 [==============================] - 182s 85ms/step - loss: 0.6949 - accuracy: 0.7076 - val_loss: 0.6293 - val_accuracy: 0.7490
Epoch 2/2
1563/1563 [==============================] - 125s 80ms/step - loss: 0.4506 - accuracy: 0.8300 - val_loss: 0.7138 - val_accuracy: 0.7373


In [18]:
print("\n 테 스 트 정 확 도 : %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)
[1]))

5/5 [==============================] - 2s 235ms/step - loss: 0.6977 - accuracy: 0.7405

 테 스 트 정 확 도 : 0.7405
